# **Melody Generation from Images**

## **Importing Libraries**

In [1]:
print("no problem")

no problem


In [34]:
from numpy import array
from pickle import load, dump
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import add
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from glob import glob
import os
from numpy import argmax
from music21 import converter, instrument, note, chord, stream, meter,duration, interval, pitch, tempo, midi
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.layers import Input, Dropout, Dense, Embedding, Add, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D, Reshape, Flatten
from os import listdir
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

**Mount Drive**

In [35]:
# from google.colab import drive
# drive.mount('/content/drive')

To include data folder into drive,
go to the below link and click on add shortcut to drive
https://drive.google.com/drive/folders/1TTg5wH78eE0yR_OHJZTJzF3wGv8t0Q1x?usp=sharing

**Declare Constants**

In [3]:
pwd

'/mnt/32624a58-179e-4823-a676-9ab25dc5b8c0b10971'

In [2]:
#data folder path
# data_dir = r'E:\uniSlides\CS\Research\FinalDataset\FinalData\unique_2VGG_FINAL\\'
data_dir = ""
#midi note and duration separator
separator = "@"
#image classifier model labels
label_VGG16 = "VGG16"
label_InceptionV3 = "InceptionV3"
#image model currently in use
current_ImgModel = label_VGG16
# current_ImgModel = label_InceptionV3

## **Midi Preprocessing**

Obtain all midi files from folder

In [3]:
# songs = glob(data_dir + 'midi/*.MID')
songs = glob('train_data/audio/*.mid')
songs.sort()
songs

[]

### **Function Definitions**

Define function to convert list to string

In [4]:
def listToString(listObj):

    # initialize an empty string
    stringText = ""

    # traverse in the string
    for element in listObj:
        stringText = stringText + " " + element

    # return string
    return stringText

Function to transpose a song to C maj if Major or A min if Minor

In [5]:
def transpose(song):
    #Transposes song to C maj/A min

    # get key using music21
    print("Ok")
    key = song.analyze("key")
    print(key,"Ok")
    # get interval for transposition. E.g., Bmaj -> Cmaj
    if key.mode == "major":
        intervalSong = interval.Interval(key.tonic, pitch.Pitch("C"))
    elif key.mode == "minor":
        intervalSong = interval.Interval(key.tonic, pitch.Pitch("A"))

    # transpose song by calculated interval
    tranposed_song = song.transpose(intervalSong)
    return tranposed_song

Function to check if rests are more than one bar length

In [6]:
#function to check if rest duration is acceptables
def isRestAcceptable(rest, timeSignature):
  #check is time signature denominator is 4
  if (timeSignature.denominator == 4):
      #if denominator 4, length of one bar is 4 beats
      barDuration = timeSignature.numerator
  #check is time signature denominator is 8
  elif (timeSignature.denominator == 8):
      #if denominator is 8, length of one bar is numerator/2
      barDuration = timeSignature.numerator/2.0
  #check is time signature denominator is 2
  elif (timeSignature.denominator == 2):
      #if denominator is 2, length of one bar is numerator * 2
      barDuration = timeSignature.numerator * 2.0
  else : barDuration = 0

  #return false if rest duration is greater than 2 bars
  if barDuration * 2 < rest.quarterLength:
      return False

  return True

Function to get bar in which 1st note occurs

In [7]:
def getStartingBar(score):
    num = -1
    for i in range(51):
        bar = score.measures(i-1,i)
        barflat = bar.flatten()
        for event in barflat:
            if isinstance(event, note.Note):
                num = i
                break
        else:
            continue
        break
    return num

Define function to create midi data file

In [8]:
def createDatafile():
  #create a file named midiData.txt and append
  midiData_file = open("midiData.txt", "w")
#iterate over each midi file
  print("Writing midi data to file...")
  songCount = 0
  for file in songs:
    print("Processing " + str(file))
    print(file)
    try:
        midi = converter.parse(file) #convert to midi
    except:
        print("Error in parsing " + str(file))
        continue
    print("midi = ",midi)
    #midi = midi.stripTies()
    #tranpose song
    try:
        midi = transpose(midi)
    except:
        print("Error in obtaining key for " + str(file))
        continue
    #obtain 30 bars
    songTempo = ''
    try:
        firstbar = midi.measures(0,3)
        firstbarNotes = firstbar.flatten()
        for element in firstbarNotes:
            if isinstance(element, tempo.MetronomeMark):
                songTempo = str(element.number) + separator + "tempo"
        startBar = getStartingBar(midi)
        midi = midi.measures(startBar, startBar + 7)
    except Exception as e:
      print("Error in measures for " + str(file))
      continue
    midi = midi.flatten() #combine all parts and get a single notes part

    songCount += 1
    #initialize timeSignature, notes list and count variables
    timeSignature = ''
    notes = []
    #count = 0

    #loop over each midi event
    #midi events include timesignatures, instruments, notes, rests and chords
    for event in midi:
      #only obtain 50 events
      # if count == 50:
      #   break;
      # count += 1 #increase count
      #check if event is a timeSignature
      if isinstance(event, meter.TimeSignature ):
        #save time signature to file as '3/4@time'
        timeSignature = event.ratioString + separator + "time"
        timeSignatureEvent = event
      #check if event is a note
      if isinstance(event, note.Note):
        #save note to file as 'C1@0.5' where 'C1' is midi note and '0.5' is the duration
        notes.append(str(event.pitch) + separator + str(event.quarterLength))
      #check if event is a chord
      elif(isinstance(event, chord.Chord)):
        #save chord to file as '1.8.2@0.5' where '1.8.2' are notes in the chord and '0.5' is the duration
        notes.append(('.'.join(str(n) for n in event.normalOrder))+ separator + str(event.quarterLength))
      #check if event is a rest
      elif(isinstance(event, note.Rest)):
        #check if rest duration is acceptable
        if isRestAcceptable(event, timeSignatureEvent):
          #save rest as 'r@0.5' where 'r' indicates that it is a rest and '0.5'  is the duration
          notes.append('r' + separator + str(event.quarterLength))

    #save the sequence to the file
    # jpgFilenamesList = glob(data_dir + 'images/' + os.path.basename(file).split('.')[0] + '*.*')
    jpgFilenamesList = ['image/' + os.path.basename(file).split('.')[0] + '.jpg']
    #jpgFilenamesList = glob(
    simImageCount = 0
    for image in jpgFilenamesList:
        simImageCount += 1
        #first write the file id which is same as the image id
        sequence = os.path.basename(image).split('.')[0] + " " + "<start> " + songTempo + " " + timeSignature + listToString(notes) + " <end>" #append <start> and <end> tokens
        #if current song is the last in the list, don't add a newline character to the end
        if((len(songs)== songs.index(file) +1) and simImageCount == len(jpgFilenamesList)):
            pass
        else:
            sequence = sequence + "\n"
        midiData_file.write(sequence)

  print(f"Finishes writing midi data. {songCount} songs written.")
  #close file
  midiData_file.close()

Function to load a file

In [9]:
def load_file(fileName):
  """

  Args:
    fileName:

  Returns:
    content:

  """
  #open file in read mode
  file = open(fileName, 'r')
  #obtain content in file
  content = file.read()
  #close file
  file.close()

  return content

Function to map file content to dict, create vocabulary and obtain max length of a midi sequence

In [10]:
def dict_vocab_maxLength(content):
  #initialize dictionary
  midiDict = dict()
  vocabulary = set()
  maxLength = 0
  #loop over each line in content -> each midi song is separated by new line
  for line in content.split('\n'):
    #split line into tokens by white space
    tokens = line.split()
    #first token is image id, rest are midi events
    imageId, midi = tokens[0], tokens[1:]
    #create a list if not created
    if imageId not in midiDict:
      midiDict[imageId] = list()
    midiDict[imageId].append(midi)
    #build vocabulary
    vocabulary.update(midi)
    #store max length
    if len(tokens) > maxLength:
      maxLength = len(tokens)
  #return dict, vocabulary and maxlength
  return midiDict, vocabulary, maxLength

Function to tokenize vocabulary and return tokenizer

In [11]:
def tokenize(vocab):
  #filter 50 midi events with low frequency
  num_words= len(vocab) - 10
  #initialize tokenizer object without any filters
  tokenizer = Tokenizer(num_words=num_words, filters='')
  #generate tokens
  tokenizer.fit_on_texts(vocab)
  print("Number of tokens: " + str(len(tokenizer.word_index)))
  return tokenizer

### **Midi Preprocessing steps**

Create a text file with all midi data of all songs

In [12]:
# createDatafile()

Load Saved file

In [13]:
content = load_file("/home/souraja/All_Data_OneAI_Souraja_Final/Image_to_Sequence/midiData.txt")

Create a Dictionary with midi data, obtain vocabulary and get the max length of a midi sequence

In [14]:
train_midi, vocabulary, maxLength = dict_vocab_maxLength(content)
#print vocabulary length
vocabSize = len(vocabulary) + 1
print("Vocabulary length = " + str(vocabSize))
#print max length of sequence
print("Maximum length sequence = " + str(maxLength))
print(train_midi['0001'])
print(vocabulary)
# maxLength

Vocabulary length = 978
Maximum length sequence = 236
[['<start>', '114.0@tempo', '4/4@time', '4@0.25', '0.4.7@0.25', 'E5@1.0', '0.4.7@1.0', '4@0.75', '4@1.0', '2@0.25', '2@0.75', '2.4.8@1.0', '2@0.25', '2.4.8@0.75', 'E5@1.0', '2.4.8@0.25', '2@0.75', '9@1.0', '0@0.25', '0@0.75', '9.0.4@1.0', '0@0.25', 'r@1.5', 'r@1.0', '9.0.4@1.0', 'A4@1/3', '5@0.75', 'D5@1/3', 'r@1/6', 'F5@0.25', 'r@2/3', 'r@0.25', '5@0.25', 'A5@1/3', '2.5.9@1.0', 'D6@1/3', 'r@1/6', 'F6@0.25', 'r@1/6', 'r@0.25', 'r@1.5', 'A6@1.0', '2.5.9@1.0', 'E4@1/3', '7@1.0', 'G4@1/3', 'r@1/6', 'C5@0.25', 'r@2/3', 'r@0.25', 'E5@1/3', '0.4.7@0.75', 'G5@1/3', 'r@1/6', 'C6@0.25', 'r@1/6', 'r@0.25', 'r@2.5', '0.4.7@0.25', 'E6@1.0', '0.4.7@1.0', 'D6@0.5', '7@1.0', 'r@0.5', 'G5@1/3', '5.7.11@1.0', 'D6@1/3', 'r@1/6', 'G6@0.25', 'r@1/6', 'r@0.25', 'r@0.5', 'G5@1/3', '5.7.11@0.75', 'D6@1/3', 'r@1/6', 'G6@0.25', 'r@13/6', 'r@0.25', '5.7.11@0.25', 'C6@1.75', 'C3@1.0', '0.4.7@1.0', 'r@1.25', 'r@1.5', 'C6@0.25', '0.4.7@1.0', 'A4@1/3', '5@0.75',

Tokenize vocabulary and save tokenizer pickle

In [17]:
# tokenizer = tokenize(vocabulary)
import tensorflow.keras
tokenizer = tokenize(vocabulary)

Number of tokens: 977


In [18]:
tokenizer.word_index

{'b6@4/3': 1,
 '10.3@0.5': 2,
 'd4@1/3': 3,
 '11@1.0': 4,
 'e4@0.75': 5,
 'e7@4/3': 6,
 'e1@3.25': 7,
 '4.6.9.11@1.0': 8,
 '1@1.5': 9,
 'c2@1/3': 10,
 '0.2.7@1.0': 11,
 'a5@0.5': 12,
 '0.4.7@3.0': 13,
 '7.10.2@0.25': 14,
 'e1@1.0': 15,
 'r@2.75': 16,
 'a6@1.0': 17,
 '0@2.75': 18,
 'f#5@1.0': 19,
 'b-2@2.0': 20,
 '143.0@tempo': 21,
 'b6@0.5': 22,
 '4.5.9.0@2.75': 23,
 'f#4@0.5': 24,
 'g#4@2.0': 25,
 'f3@2.0': 26,
 'a1@1/3': 27,
 'b4@0.25': 28,
 'e-3@1.75': 29,
 'f#4@1.0': 30,
 'd6@0.5': 31,
 '7.0@1/3': 32,
 'g4@0.5': 33,
 '7@0.75': 34,
 '1.4.8@1.0': 35,
 'b-4@0.5': 36,
 'a4@1.0': 37,
 '1.6@0.5': 38,
 '30.0@tempo': 39,
 '4.7.11@2.0': 40,
 'e-5@0.25': 41,
 'a-5@0.25': 42,
 '8.0@0.5': 43,
 'f2@2.0': 44,
 'b-4@0.75': 45,
 'b3@1/12': 46,
 '5.9.0@0.25': 47,
 'e6@1/12': 48,
 'b3@1.5': 49,
 'b-3@0.5': 50,
 '0.5@2.0': 51,
 '6.9@1.0': 52,
 'b-2@0.5': 53,
 '9.11.4@4.0': 54,
 '2.5@1/12': 55,
 'c2@3.25': 56,
 '5@2.0': 57,
 'e5@0.25': 58,
 'g3@2.0': 59,
 'a4@0.75': 60,
 '0.2.7@1/3': 61,
 '6.9.0@1.0':

In [19]:
# dump(tokenizer, open('tokenizer.pkl', 'wb'))

## **Image Preprocessing**

### **Function Definitions**

Function to extract image features

In [19]:
def extract_Image_features(directory, modelName = current_ImgModel):
  #check for the model needed and initialize the model and image size
  if modelName == label_VGG16:
    model = VGG16()
    imgSize = (224, 224)
  else:
    model = InceptionV3(weights='imagenet')
    imgSize = (299, 299)

  #remove the last layer of the model to obtain the features
  #VGG16 has 4096 and InceptionV3 has 2048
  model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

  #print model summary
  print(model.summary())

  #intialize dictionary to extract features
  features = dict()

  #loop over each image in directory
  for imgName in listdir(directory):
    #get image path
    path = directory + '/' + imgName
    #load image and resize
    image = load_img(path, target_size=imgSize)
    #convert image to numpy array
    image = img_to_array(image)
    #reshape image to suit model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    #preprocess image for the model
    image = preprocess_input(image)
    #extract features
    feature = model.predict(image, verbose=0)
    #get image id
    image_id = imgName.split('.')[0]
    #append to dictionary
    features[image_id] = feature

    #print image name
    print("Features extracted for " + imgName)

  return features

Function to load photo features from pickle file

In [20]:
def load_featuresPickle(path):
  #load all features
  features = load(open(path, 'rb'))
  return features

### Image Preprocessing Steps

In [22]:
# features = extract_Image_features('image')

Dump feature to pickle file

In [23]:
# #print length of features
# print("Extracted features : %d" % len(features))
# # save to pickle file
# dump(features, open('features.pkl', 'wb'))

# **Model Creation, Training and related Functions**

## **Function Definitions**

Function to create a deep learning model

In [21]:
from keras.layers import Input, Dropout, Dense, Embedding, Add, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from keras.models import Model
from keras.optimizers import Adam

def create_model_with_2_transformer_VA_loss(vocab_size, max_length, feature_extractor_label="InceptionV3"):
    # Image feature extractor model
    if feature_extractor_label == "VG16":
        inputs1 = Input(shape=(4096,))
    elif feature_extractor_label == "InceptionV3":
        inputs1 = Input(shape=(2048,))

    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # Sequence model with Transformer
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.2)(se1)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(se2, se2)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    se3 = GlobalAveragePooling1D()(attention_output)

    # Decoder model
    decoder1 = Add()([fe2, se3])
    decoder2 = Dense(128, activation='relu')(decoder1)
    
    decoder3 = Dense(256, activation='relu')(decoder2)
    outputs = Dense(vocab_size, activation='softmax')(decoder3)

    # Merging model
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss=custom_loss, optimizer=Adam(learning_rate=0.0001))

    # Print model summary
    model.summary()

    return model

In [25]:
def create_model_with_2_transformer(vocab_size, max_length, feature_extractor_label="InceptionV3"):
    # Image feature extractor model
    if feature_extractor_label == "VG16":
        inputs1 = Input(shape=(4096,))
    elif feature_extractor_label == "InceptionV3":
        inputs1 = Input(shape=(2048,))

    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # Sequence model with Transformer
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.2)(se1)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(se2, se2)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    se3 = GlobalAveragePooling1D()(attention_output)

    # Decoder model
    decoder1 = Add()([fe2, se3])
    decoder2 = Dense(128, activation='relu')(decoder1)
    
    decoder3 = Dense(256, activation='relu')(decoder2)
    outputs = Dense(vocab_size, activation='softmax')(decoder3)

    # Merging model
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001))

    # Print model summary
    model.summary()

    return model

In [22]:
from functools import partial
import tensorflow as tf
def create_Model(vocabSize, maxLength):
  #image feature extractor model
  if current_ImgModel == label_VGG16:
    inputs1 = Input(shape = (4096, ))
  elif current_ImgModel == label_InceptionV3:
    inputs1 = Input(shape = (2048, ))

  fe1 = Dropout(0.5)(inputs1)
  fe2 = Dense(256, activation='relu')(fe1)

  #midi sequence model
  inputs2 = Input(shape=(maxLength,))
  #Embedding layer
  se1 = Embedding(vocabSize, 256, mask_zero=True)(inputs2)
  se2 = Dropout(0.2)(se1)
  se3 = LSTM(128, return_sequences=True)(se2)
  se3 = LSTM(256, return_sequences=True)(se3)
  se4 = Dropout(0.2)(se3)
  se5 = LSTM(256)(se4)

  #decoder model
  decoder1 = add([fe2, se5])
  decoder2 = Dense(128, activation='relu')(decoder1)
  decoder2 = Dense(256, activation='relu')(decoder2)
  outputs = Dense(vocabSize, activation='softmax')(decoder2)

  #Merger model
  model = Model(inputs=[inputs1, inputs2], outputs=outputs)
  #compile model
  model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001))
  #print model summary
  print("Model Summary")
  model.summary()
  # plot_model(model, to_file='model.png', show_shapes=True)
  return model

Function to generate input sequences

In [23]:
def create_sequences(tokenizer, max_length, midi_list, photo, vocab_size):
  #initialize input lists
  X1, X2, y = list(), list(), list()
  #loop through each midi song for the image
  for midi in midi_list:
    #encode the midi sequence
    seq = tokenizer.texts_to_sequences([midi])[0]
    #generate multiple X,y pairs from one midi file
    for i in range(1, len(seq)):
      #generate input and output pair
      in_seq, out_seq = seq[:i], seq[i]
      #pad input sequence
      in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
      #encode output Sequence
      out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
      #append to input lists
      X1.append(photo)
      X2.append(in_seq)
      y.append(out_seq)

  return array(X1), array(X2), array(y)

Function to generate data for progressive loading

In [24]:
def data_generator(midiData, photos, tokenizer, max_length, vocab_size):
  #loop forever
  while True:
    for key, midi_list in midiData.items():
      #get photo features
      photo = photos[key][0]
      in_img, in_seq, out_word = create_sequences(tokenizer, max_length, midi_list, photo, vocab_size)
      # yield [in_img, in_seq], out_word
      # print("in_img = ",in_img)
      # print("in_seq = ",in_seq)
      # print("out_word = ",out_word)
      yield (in_img, in_seq), out_word

Function to map output integer to midi vocabulary

In [25]:
def get_midiString_for_Integer(integer, tokenizer):
  #loop through tokenizer to find a match
  for midiString, index in tokenizer.word_index.items():
    if index == integer:
      return midiString
  return None

Function to generate midi string for image

In [30]:
tokenizer.texts_to_sequences(['<start> 85@tempo 4/4@time G4@2.75 0.4.7@2.75','<start>'])

[[291, 434, 193, 651], [291]]

In [26]:
def generate_midiSequence(model, tokenizer, photo, maxLength):
  #create initial token
  midiSequence = '<start>' #string with midi events including <start> and <end> tokens
  prediction_list = [] #list with midi events without <start> and <end> tokens

  #iterate over max length of a sequence
  for i in range(maxLength):
    #encode sequence
    sequence =  tokenizer.texts_to_sequences([midiSequence])[0]
    #pad sequence
    sequence = pad_sequences([sequence], maxlen=maxLength)
    #predict next midi event string
    yhat = model.predict([photo,sequence], verbose=0)
    #print(yhat)
    #obtain event with highest probability
    #import numpy as np
    #yhat2 = np.argsort(np.max(yhat, axis=0))[-2]
    yhat = argmax(yhat)
    #map integer to midi string
    midiString = get_midiString_for_Integer(yhat, tokenizer)
    #stop if cannot find
    if midiString is None:
      break
    #append midiString to sequence
    if not midiString == '<start>':
        midiSequence += ' ' + midiString
    #stop if end of midi
    if midiString == '<end>':
      break
    #append midi event string to prediction list
    if not midiString == '<start>':
        prediction_list.append(midiString)

  return midiSequence, prediction_list

Function to create a midi from prediction list

In [27]:
def create_midi(prediction_output, midiName):
  #initiate offset to 0
  offset = 0.0
  #initiate midi stram
  midi_stream = stream.Stream()

  #loop over each midiString patter in prediction Output
  for pattern in prediction_output:
    #Seperate midiString into event and time by @ symbol
    patternString = pattern.split('@')[0] #0 position stores midi event
    if pattern.split('@')[1] == 'tempo':
      #if event is a tempo, append to midiStream and continue to next iteration
      tp0 = tempo.MetronomeMark(patternString)
      tp0.setQuarterBPM(int(float(patternString)))
      midi_stream.append(tp0)
      continue
    #check if position 1 == time to check if event is a timeSignature
    if pattern.split('@')[1] == 'time':
      #if event is a timeSignature, append to midiStream and continue to next iteration
      ts0 = meter.TimeSignature(patternString)
      midi_stream.append(ts0)
      continue
    #check if '.' in patterString  or it patternString is a digit to detemine if the event is a chord
    if ('.' in patternString) or patternString.isdigit():
        #if event is a chord, obtain notes
        notes_in_chord = patternString.split('.')
        notes = []
        #loop for each note and create a notes list
        for current_note in notes_in_chord:
          new_note = note.Note(int(current_note))
          new_note.storedInstrument = instrument.Piano()
          notes.append(new_note)
        #create a chord using notes list
        new_chord = chord.Chord(notes)
        #set duration of chord
        new_chord.quarterLength = eval(pattern.split('@')[1])
        #update offset
        new_chord.offset = offset
        offset += new_chord.quarterLength
        #append chord to midi Stream
        midi_stream.append(new_chord)
    # pattern is a note or rest
    else:
      #if pattern is a rest
      if patternString == 'r':
        #create rest event
        new_note = note.Rest()
        #set duration
        new_note.quarterLength = eval(pattern.split('@')[1])
        #update offest
        new_note.offset = offset
        offset += new_note.quarterLength
        #append to midi stream
        midi_stream.append(new_note)
      else:
        #if pattern is a note
        #create note
        new_note = note.Note(patternString)
        #set note duration
        new_note.quarterLength = eval(pattern.split('@')[1])
        #update offset
        new_note.offset = offset
        offset += new_note.quarterLength
        new_note.storedInstrument = instrument.Piano()
        midi_stream.append(new_note)

  midi_stream.makeMeasures(inPlace = True)

  print('Saving Output file as midi....')

  midi_stream.write('midi', fp='testMidi_trans_incept/' + midiName + '.mid')

Function to extract features of image to generate music

In [28]:
def extract_featuresPredict(path, modelName = current_ImgModel):
  #initialize model and resize image
  if modelName == label_VGG16:
    model = VGG16()
    image = load_img(path, target_size=(224, 224))
  elif modelName == label_InceptionV3:
    model = InceptionV3(weights='imagenet')
    image = load_img(path, target_size=(299, 299))

  #remove last layer of model
  model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
  #convert image to numpy array
  image = img_to_array(image)
  #reshape array
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  #preprocess image
  image = preprocess_input(image)
  #obtain features
  features = model.predict(image, verbose=0)

  return features

Model Creation and Training Steps

Load training photo features

In [34]:
train_features = load_featuresPickle('features_incept.pkl')
# train_features = load_featuresPickle('features.pkl')
print(len(train_features))

3000


Training midi data is already loaded in train_midi dictionary

Next create the model

In [34]:
#initialize epochs
epochs = 250
#initials steps to generate data
steps = len(train_midi)
#create model
# model = create_Model(vocabSize= vocabSize, maxLength=maxLength)
model = create_model_with_transformer(vocab_size= vocabSize, max_length=maxLength)


NameError: name 'create_model_with_transformer' is not defined

Define path to save best model and initialize Checkpoint

In [33]:
#save weights
# filepath = 'training/cp.ckpt'
filepath = 'training_new_model/cp.weights.h5'
checkpoint_dir = os.path.dirname(filepath)
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

Train model

In [36]:
print(len(train_features['0001'][0]))
print(train_features['0002'][0])
print(train_features['0003'][0])
print(train_features['0101'][0])
print(train_features['0301'][0])

4096
[0.         0.59926796 0.         ... 2.9459355  0.         0.4254136 ]
[2.0397887  0.         0.72674024 ... 0.6001403  0.         3.0588882 ]
[0.        3.2399967 0.        ... 0.        1.675365  0.9805833]
[5.0451164  0.         0.23566201 ... 0.         0.         1.1905509 ]


In [37]:
# for i in range(epochs):
#   #create data generator
#   generator  = data_generator(train_midi, train_features, tokenizer, maxLength, vocabSize)
#   #fit model for one epoch
#   print("Epoch No : " + str(i + 1))
#   model.fit(generator, epochs=1, steps_per_epoch=steps, callbacks=[checkpoint], verbose=1)

In [ ]:
# #resume training
# #load weights
# filepath = 'training/cp.ckpt'
# checkpoint_dir = os.path.dirname(filepath)
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

In [ ]:
# model = create_Model(vocabSize, maxLength)
# model.load_weights(filepath)
# tokenizer = load(open(data_dir + 'tokenizer.pkl', 'rb'))

In [ ]:
# for i in range(epochs):
#   #create data generator
#   generator = data_generator(train_midi, train_features, tokenizer, maxLength, vocabSize)
#   #fit model for one epoch
#   print("Epoch No : " + str(i + 1))
#   model.fit(generator, epochs=1, steps_per_epoch=steps, callbacks=[checkpoint], verbose=1)

In [ ]:
# #resume training
# #load weights
# filepath = data_dir + 'training_2/cp.ckpt'
# checkpoint_dir = os.path.dirname(filepath)
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

In [ ]:
# model = create_Model(vocabSize, maxLength)
# model.load_weights(filepath)
# tokenizer = load(open(data_dir + 'tokenizer.pkl', 'rb'))

Model Summary
Model: "model_50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_72 (InputLayer)          [(None, 114)]        0           []                               
                                                                                                  
 embedding_20 (Embedding)       (None, 114, 256)     87040       ['input_72[0][0]']               
                                                                                                  
 dropout_61 (Dropout)           (None, 114, 256)     0           ['embedding_20[0][0]']           
                                                                                                  
 input_71 (InputLayer)          [(None, 4096)]       0           []                               
                                                                             

In [ ]:
# for i in range(epochs):
#   #create data generator
#   generator = data_generator(train_midi, train_features, tokenizer, maxLength, vocabSize)
#   #fit model for one epoch
#   print("Epoch No : " + str(i + 1))
#   model.fit(generator, epochs=1, steps_per_epoch=steps, callbacks=[checkpoint], verbose=1)

Epoch No : 1
300/300 [==============================] - ETA: 0s - loss: 0.1251
Epoch 1: loss improved from inf to 0.12512, saving model to E:\uniSlides\CS\Research\FinalDataset\FinalData\unique_2VGG_FINAL\\training_2\cp.ckpt
300/300 [==============================] - 958s 3s/step - loss: 0.1251
Epoch No : 2
300/300 [==============================] - ETA: 0s - loss: 0.1025
Epoch 1: loss improved from 0.12512 to 0.10249, saving model to E:\uniSlides\CS\Research\FinalDataset\FinalData\unique_2VGG_FINAL\\training_2\cp.ckpt
300/300 [==============================] - 871s 3s/step - loss: 0.1025
Epoch No : 3
300/300 [==============================] - ETA: 0s - loss: 0.1026
Epoch 1: loss did not improve from 0.10249
300/300 [==============================] - 885s 3s/step - loss: 0.1026
Epoch No : 4
300/300 [==============================] - ETA: 0s - loss: 0.1393
Epoch 1: loss did not improve from 0.10249
300/300 [==============================] - 775s 3s/step - loss: 0.1393
Epoch No : 5
191/3

KeyboardInterrupt: 

In [66]:
def create_model_with_4_transformer_MIDI(vocab_size, max_length, feature_extractor_label="VG16"):
    # Image feature extractor model
    if feature_extractor_label == "VG16":
        inputs1 = Input(shape=(4096,))
    elif feature_extractor_label == "InceptionV3":
        inputs1 = Input(shape=(2048,))

    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # Sequence model with Transformer
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.2)(se1)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(se2, se2)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    se3 = GlobalAveragePooling1D()(attention_output)

    # Decoder model
    decoder1 = Add()([fe2, se3])
    decoder2 = Dense(128, activation='relu')(decoder1)
    decoder3 = Dense(256, activation='relu')(decoder2)
    outputs = Dense(vocab_size, activation='softmax')(decoder3)

    # Merging model
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001))

    # Print model summary
    model.summary()

    return model

In [42]:
def create_model_with_4_transformer_decoder_MIDI(vocab_size, max_length, feature_extractor_label="VG16"):
    # Image feature extractor model
    if feature_extractor_label == "VG16":
        inputs1 = Input(shape=(4096,))
    elif feature_extractor_label == "InceptionV3":
        inputs1 = Input(shape=(2048,))

    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # Sequence model with Transformer
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.2)(se1)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(se2, se2)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    se3 = GlobalAveragePooling1D()(attention_output)

    # Decoder model
    decoder1 = Add()([fe2, se3])
    transdec1 = MultiHeadAttention(num_heads=4, key_dim=256)(decoder1, decoder1)
    transdec2 = LayerNormalization(epsilon=1e-6)(transdec1)
    transdec3 = Dropout(0.2)(transdec2)
    
    decoder2 = Dense(128, activation='relu')(transdec3)
    decoder3 = Dense(256, activation='relu')(decoder2)
    outputs = Dense(vocab_size, activation='softmax')(decoder3)

    # Merging model
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001))

    # Print model summary
    model.summary()

    return model

In [116]:
def create_model_with_transformer_enc2_dec2(vocab_size, max_length, feature_extractor_label="InceptionV3"):
    # Image feature extractor model
    if feature_extractor_label == "VG16":
        inputs1 = Input(shape=(4096,))
    elif feature_extractor_label == "InceptionV3":
        inputs1 = Input(shape=(2048,))
    
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    
    # Sequence model with Transformer
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.2)(se1)
    
    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(se2, se2)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)
    
    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)
    
    se3 = GlobalAveragePooling1D()(attention_output)
    
    # Decoder model
    decoder1 = Add()([fe2, se3])
    
    # Adding first transformer decoder layer
    decoder_attention1 = MultiHeadAttention(num_heads=4, key_dim=256)(Reshape((1, 256))(decoder1), Reshape((1, 256))(decoder1))
    decoder_attention1 = Flatten()(decoder_attention1)
    decoder_attention1 = LayerNormalization(epsilon=1e-6)(decoder_attention1)
    decoder_attention1 = Dropout(0.2)(decoder_attention1)
    
    decoder2 = Dense(128, activation='relu')(decoder_attention1)
    
    # Adding second transformer decoder layer
    decoder_attention2 = MultiHeadAttention(num_heads=4, key_dim=256)(Reshape((1, 128))(decoder2), Reshape((1, 128))(decoder2))
    decoder_attention2 = Flatten()(decoder_attention2)
    decoder_attention2 = LayerNormalization(epsilon=1e-6)(decoder_attention2)
    decoder_attention2 = Dropout(0.2)(decoder_attention2)
    
    decoder3 = Dense(256, activation='relu')(decoder_attention2)
    outputs = Dense(vocab_size, activation='softmax')(decoder3)
    
    # Merging model
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001))
    
    # Print model summary
    model.summary()
    
    return model

In [35]:
def create_model_with_3_enc_dec_transformer(vocab_size, max_length, feature_extractor_label="VG16"):
    # Image feature extractor model
    if feature_extractor_label == "VG16":
        inputs1 = Input(shape=(4096,))
    elif feature_extractor_label == "InceptionV3":
        inputs1 = Input(shape=(2048,))

    
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    
    # Sequence model with Transformer
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.2)(se1)
    
    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(se2, se2)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)
    
    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)

    attention_output = MultiHeadAttention(num_heads=4, key_dim=256)(attention_output, attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    attention_output = Dropout(0.2)(attention_output)
    
    se3 = GlobalAveragePooling1D()(attention_output)
    
    # Decoder model
    decoder1 = Add()([fe2, se3])
    
    # Adding first transformer decoder layer
    decoder_attention1 = MultiHeadAttention(num_heads=4, key_dim=256)(Reshape((1, 256))(decoder1), Reshape((1, 256))(decoder1))
    decoder_attention1 = Flatten()(decoder_attention1)
    decoder_attention1 = LayerNormalization(epsilon=1e-6)(decoder_attention1)
    decoder_attention1 = Dropout(0.2)(decoder_attention1)
    
    decoder2 = Dense(128, activation='relu')(decoder_attention1)
    
    # Adding second transformer decoder layer
    decoder_attention2 = MultiHeadAttention(num_heads=4, key_dim=256)(Reshape((1, 128))(decoder2), Reshape((1, 128))(decoder2))
    decoder_attention2 = Flatten()(decoder_attention2)
    decoder_attention2 = LayerNormalization(epsilon=1e-6)(decoder_attention2)
    decoder_attention2 = Dropout(0.2)(decoder_attention2)

    decoder3 = Dense(128, activation='relu')(decoder_attention2)

    # Adding third transformer decoder layer
    decoder_attention3 = MultiHeadAttention(num_heads=4, key_dim=256)(Reshape((1, 128))(decoder3), Reshape((1, 128))(decoder3))
    decoder_attention3 = Flatten()(decoder_attention3)
    decoder_attention3 = LayerNormalization(epsilon=1e-6)(decoder_attention3)
    decoder_attention3 = Dropout(0.2)(decoder_attention3)
    
    decoder4 = Dense(256, activation='relu')(decoder_attention3)
    outputs = Dense(vocab_size, activation='softmax')(decoder4)
    
    # Merging model
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001))
    
    # Print model summary
    model.summary()
    plot_model(model, to_file = 'trans-enc3_dec3.png', show_shapes = True, show_layer_names = True)
    
    return model

In [38]:
# Define your custom loss function
def custom_loss(y_true, y_pred, weight=0.5):
  # Assuming your tensors are named 'compile_loss/custom_loss/Cast' and 'functional_1/dense_3_1/Softmax'
  with tf.device("/CPU:0"):  # Or specify your desired device
      # print(y_true.numpy().shape)
      # print(y_pred.numpy().shape)

      loss_values = y_true
      prediction_values = y_pred

      midiSequence_true = '<start>' #string with midi events including <start> and <end> tokens
      prediction_list_true = [] #list with midi events without <start> and <end> tokens


      # Loop through the elements
      for i in range(len(loss_values)):
          ele = loss_values[i]
          ele = argmax(ele)
          #map integer to midi string
          midiString_true = get_midiString_for_Integer(ele, tokenizer)
          #stop if cannot find
          if midiString_true is None:
            break
          #append midiString to sequence
          if not midiString_true == '<start>':
              midiSequence_true += ' ' + midiString_true
          #stop if end of midi
          if midiString_true == '<end>':
            break
          #append midi event string to prediction list
          if not midiString_true == '<start>':
              prediction_list_true.append(midiString_true)

      midiSequence_pred = '<start>' #string with midi events including <start> and <end> tokens
      prediction_list_pred = [] #list with midi events without <start> and <end> tokens
      # Loop through the elements
      for i in range(len(loss_values)):
          ele = prediction_values[i]
          ele = argmax(ele)
          #map integer to midi string
          midiString_pred = get_midiString_for_Integer(ele, tokenizer)
          #stop if cannot find
          if midiString_pred is None:
            break
          #append midiString to sequence
          if not midiString_pred == '<start>':
              midiSequence_pred += ' ' + midiString_pred
          #stop if end of midi
          if midiString_pred == '<end>':
            break
          #append midi event string to prediction list
          if not midiString_pred == '<start>':
              prediction_list_pred.append(midiString_pred)

  
  create_midi(prediction_list_true, 'tmp_true')
  create_midi(prediction_list_pred, 'tmp_pred')
  true_music = muspy.read_midi('testMidi_trans3_VA_loss/tmp_true.mid')
  pred_music = muspy.read_midi('testMidi_trans3_VA_loss/tmp_pred.mid')
  rep_true = muspy.to_note_representation(true_music)
  rep_pred = muspy.to_note_representation(pred_music)
  rep_true = np.pad(rep_true, ((0, 500-rep_true.shape[0]), (0, 0)), 'constant', constant_values=1)
  rep_pred = np.pad(rep_pred, ((0, 500-rep_pred.shape[0]), (0, 0)), 'constant', constant_values=1)
  rep_true = rep_true.reshape(1, 2000)
  rep_pred = rep_pred.reshape(1, 2000)


  filepath = 'music.keras'
  # model = tf.keras.Sequential([
  #    Dense(1024, activation = 'relu'),
  #    Dense(512, activation = 'relu'),
  #    Dense(2, activation = 'relu'),
  # ])
  # model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001))

  # model = tf.keras.models.model_from_config(config)
  model = tf.keras.models.load_model(filepath)
  # converter = tf.lite.TFLiteConverter.from_keras_model(model)
  # tflitemodel = converter.convert()
  # with open('converted_model.tflite','wb') as f:
  #     f.write(tflitemodel)
  # batch_size = 1
  # model.input.set_shape((batch_size,) + model.input.shape[1:])
  VA_true = model.predict(rep_true)
  VA_pred = model.predict(rep_pred)
  new_loss = (VA_true - VA_pred)**2


  # Combine existing categorical crossentropy loss with your new loss
  categorical_crossentropy = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
  total_loss = weight * new_loss + (1 - weight) * categorical_crossentropy
  return total_loss

In [36]:
model = create_model_with_3_enc_dec_transformer(vocab_size= vocabSize, max_length=maxLength)

2024-07-16 14:01:41.106396: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: UNKNOWN ERROR (34)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 236)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 236, 256)             250368    ['input_2[0][0]']             
                                                                                                  
 dropout_1 (Dropout)         (None, 236, 256)             0         ['embedding[0][0]']           
                                                                                                  
 multi_head_attention (Mult  (None, 236, 256)             1051904   ['dropout_1[0][0]',           
 iHeadAttention)                                                     'dropout_1[0][0]']       

In [37]:
!pip install pydot

Could not fetch URL https://pypi.org/simple/pydot/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pydot/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1131)'))) - skipping
ERROR: Could not find a version that satisfies the requirement pydot (from versions: none)
ERROR: No matching distribution found for pydot
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1131)'))) - skipping


Generate a melody for a test image

In [35]:
import tensorflow as tf
import time
#load tokenizer from pickle
# tokenizer = load(open('tokenizer.pkl', 'rb'))
#load best model
# model = create_Model(vocabSize, maxLength)
model = create_model_with_2_transformer(vocab_size= vocabSize, max_length=maxLength)
# filepath = 'training/cp.weights.h5'
filepath = 'training_new_model/transincept_enc2.weights.h5'
# model.load_weights(filepath)
# give the path to testImages, here testImages = image
images = glob('Test_Images_100/*.jpg')
existing = glob('testMidi_trans_incept/*.mid')
# images = glob('test/image/*.jpg')
print(len(existing))
prv =0
ii = 0
prvtxt = ""
cnt = 0
timetkn = 0
for img in images:
    if f"testMidi_trans_incept/{img.split('.')[0].rsplit('/')[-1]}.mid" not in existing:
        tic = time.time()
        photo = extract_featuresPredict(img)
        #generate midi
        midiSequence, prediction_output = generate_midiSequence(model, tokenizer, photo, maxLength)
        #create midi file
        create_midi(prediction_output, img.split('.')[0].rsplit('/')[-1])
        toc = time.time()
        timetkn+=toc-tic
        if(ii > 0) : 
            print(sum((photo - prv)[0]))
            prv = photo
        ii += 1
        print("Midi Sequnce in text")
        print(midiSequence)
        print(prvtxt == midiSequence)
        prvtxt = midiSequence
    
        
        cnt+=1
        if cnt==10:
            break
print(timetkn)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 236)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 236, 256)             250368    ['input_4[0][0]']             
                                                                                                  
 dropout_5 (Dropout)         (None, 236, 256)             0         ['embedding_1[0][0]']         
                                                                                                  
 multi_head_attention_2 (Mu  (None, 236, 256)             1051904   ['dropout_5[0][0]',           
 ltiHeadAttention)                                                   'dropout_5[0][0]']     

In [38]:
existing = glob('test_midi_trans_enc2_dec2/test_midis_100/*.mid')
print(len(existing))


100


In [32]:
import tensorflow as tf
#load tokenizer from pickle
tokenizer = load(open('tokenizer.pkl', 'rb'))
#load best model
# model = create_Model(vocabSize, maxLength)
model = create_model_with_transformer(vocab_size= vocabSize, max_length=maxLength)
# filepath = 'training/cp.weights.h5'
filepath = 'training_new_model/transend.weights.h5'
model.load_weights(filepath)
# give the path to testImages, here testImages = image
images = glob('test/emotional_img/*.jfif')
# images = glob('test/image/*.jpg')
print(images)
prv =0
ii = 0
prvtxt = ""
for img in images:
    
    photo = extract_featuresPredict(img)
    #generate midi
    midiSequence, prediction_output = generate_midiSequence(model, tokenizer, photo, maxLength)
    if(ii > 0) : 
        print(sum((photo - prv)[0]))
        prv = photo
    ii += 1
    print("Midi Sequnce in text")
    print(midiSequence)
    print(prvtxt == midiSequence)
    prvtxt = midiSequence

    #create midi file
    create_midi(prediction_output, img.split('.')[0].rsplit('/')[-1])

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 236)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 236, 256)             250368    ['input_10[0][0]']            
                                                                                                  
 dropout_11 (Dropout)        (None, 236, 256)             0         ['embedding_2[0][0]']         
                                                                                                  
 multi_head_attention_7 (Mu  (None, 236, 256)             1051904   ['dropout_11[0][0]',          
 ltiHeadAttention)                                                   'dropout_11[0][0]']    

['test/emotional_img/sad_img.jfif', 'test/emotional_img/happy_img.jfif', 'test/emotional_img/angry_img.jfif', 'test/emotional_img/calm_img.jfif']
Midi Sequnce in text
<start> 50.0@tempo 4/4@time r@3.0 r@3.0 r@3.0 a3@0.5 e4@0.5 a4@0.5 c5@0.5 b4@0.5 a4@0.5 e6@1.0 a3@0.5 e4@0.5 c6@0.5 a4@0.5 d6@1.0 c5@0.5 b4@0.5 e-6@0.5 a4@0.5 e6@2.0 a3@0.5 e4@0.5 a4@0.5 c5@0.5 a5@1.0 b4@0.5 a4@0.5 f6@0.5 d3@0.5 r@2.5 r@3.5 <end>
False
Saving Output file as midi....
2863.9692875146866
Midi Sequnce in text
<start> 50.0@tempo 4/4@time r@3.0 r@3.0 r@3.0 a3@0.5 e4@0.5 a4@0.5 c5@0.5 b4@0.5 a4@0.5 e6@1.0 a3@0.5 e4@0.5 c6@0.5 a4@0.5 d6@1.0 c5@0.5 b4@0.5 e-6@0.5 a4@0.5 e6@2.0 a3@0.5 e4@0.5 a4@0.5 c5@0.5 a5@1.0 b4@0.5 a4@0.5 f6@0.5 d3@0.5 r@2.5 r@3.5 <end>
True
Saving Output file as midi....
-1268.005207195878
Midi Sequnce in text
<start> 50.0@tempo 4/4@time r@3.0 r@3.0 r@3.0 a3@0.5 e4@0.5 a4@0.5 c5@0.5 b4@0.5 a4@0.5 e6@1.0 a3@0.5 e4@0.5 c6@0.5 a4@0.5 d6@1.0 c5@0.5 b4@0.5 e-6@0.5 a4@0.5 e6@2.0 a3@0.5 e4@0.5 a4@0.5

In [58]:
current_ImgModel = label_InceptionV3

import tensorflow as tf
#load tokenizer from pickle
tokenizer = load(open('tokenizer.pkl', 'rb'))
#load best model
# model = create_Model(vocabSize, maxLength)
model = create_model_with_transformer(vocab_size= vocabSize, max_length=maxLength)
# filepath = 'training/cp.weights.h5'
filepath = 'training_new_model/transincept.weights.h5'
model.load_weights(filepath)
# give the path to testImages, here testImages = image
images = glob('test/image/*.jpg')
print(images)
prv =0
ii = 0
prvtxt = ""
for img in images:
    photo = extract_featuresPredict(img)
    #generate midi
    midiSequence, prediction_output = generate_midiSequence(model, tokenizer, photo, maxLength)
    if(ii > 0) : 
        print(sum((photo - prv)[0]))
        prv = photo
    ii += 1
    print("Midi Sequnce in text")
    print(midiSequence)
    print(prvtxt == midiSequence)
    prvtxt = midiSequence

    #create midi file
    create_midi(prediction_output, img.split('.')[0].rsplit('/')[-1])

Model: "model_65"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_74 (InputLayer)       [(None, 236)]                0         []                            
                                                                                                  
 embedding_7 (Embedding)     (None, 236, 256)             250368    ['input_74[0][0]']            
                                                                                                  
 dropout_37 (Dropout)        (None, 236, 256)             0         ['embedding_7[0][0]']         
                                                                                                  
 multi_head_attention_22 (M  (None, 236, 256)             1051904   ['dropout_37[0][0]',          
 ultiHeadAttention)                                                  'dropout_37[0][0]']   

['test/image/0050.jpg', 'test/image/0069.jpg', 'test/image/0090.jpg', 'test/image/0060.jpg', 'test/image/0074.jpg', 'test/image/0012.jpg', 'test/image/0031.jpg', 'test/image/0044.jpg', 'test/image/0087.jpg', 'test/image/0058.jpg', 'test/image/0011.jpg', 'test/image/0053.jpg', 'test/image/0003.jpg', 'test/image/0008.jpg', 'test/image/0009.jpg', 'test/image/0047.jpg', 'test/image/0019.jpg', 'test/image/0059.jpg', 'test/image/0072.jpg', 'test/image/0100.jpg', 'test/image/0057.jpg', 'test/image/0049.jpg', 'test/image/0089.jpg', 'test/image/0088.jpg', 'test/image/0056.jpg', 'test/image/0097.jpg', 'test/image/0007.jpg', 'test/image/0045.jpg', 'test/image/0017.jpg', 'test/image/0015.jpg', 'test/image/0034.jpg', 'test/image/0005.jpg', 'test/image/0054.jpg', 'test/image/0010.jpg', 'test/image/0022.jpg', 'test/image/0063.jpg', 'test/image/0098.jpg', 'test/image/0040.jpg', 'test/image/0020.jpg', 'test/image/0065.jpg', 'test/image/0036.jpg', 'test/image/0082.jpg', 'test/image/0035.jpg', 'test/imag

In [59]:
current_ImgModel = label_InceptionV3

import tensorflow as tf
#load tokenizer from pickle
tokenizer = load(open('tokenizer.pkl', 'rb'))
#load best model
# model = create_Model(vocabSize, maxLength)
model = create_model_with_transformer(vocab_size= vocabSize, max_length=maxLength)
# filepath = 'training/cp.weights.h5'
filepath = 'training_new_model/transincept.weights.h5'
model.load_weights(filepath)
# give the path to testImages, here testImages = image
images = glob('test/emotional_img/*.jfif')
print(images)
prv =0
ii = 0
prvtxt = ""
for img in images:
    photo = extract_featuresPredict(img)
    #generate midi
    midiSequence, prediction_output = generate_midiSequence(model, tokenizer, photo, maxLength)
    if(ii > 0) : 
        print(sum((photo - prv)[0]))
        prv = photo
    ii += 1
    print("Midi Sequnce in text")
    print(midiSequence)
    print(prvtxt == midiSequence)
    prvtxt = midiSequence

    #create midi file
    create_midi(prediction_output, img.split('.')[0].rsplit('/')[-1])

Model: "model_166"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_176 (InputLayer)      [(None, 236)]                0         []                            
                                                                                                  
 embedding_8 (Embedding)     (None, 236, 256)             250368    ['input_176[0][0]']           
                                                                                                  
 dropout_41 (Dropout)        (None, 236, 256)             0         ['embedding_8[0][0]']         
                                                                                                  
 multi_head_attention_24 (M  (None, 236, 256)             1051904   ['dropout_41[0][0]',          
 ultiHeadAttention)                                                  'dropout_41[0][0]']  

['test/emotional_img/sad_img.jfif', 'test/emotional_img/happy_img.jfif', 'test/emotional_img/angry_img.jfif', 'test/emotional_img/calm_img.jfif']
Midi Sequnce in text
<start> 50.0@tempo 4/4@time r@3.0 r@3.0 r@3.0 a3@0.5 e4@0.5 c5@0.5 b4@0.5 e6@1.0 a3@0.5 e4@0.5 c6@0.5 d6@1.0 c5@0.5 b4@0.5 e-6@0.5 e6@2.0 a3@0.5 e4@0.5 c5@0.5 a5@1.0 b4@0.5 f6@0.5 d3@0.5 r@2.5 r@3.5 <end>
False
Saving Output file as midi....
7520.908177367412
Midi Sequnce in text
<start> 50.0@tempo 4/4@time r@3.0 r@3.0 r@3.0 a3@0.5 e4@0.5 c5@0.5 b4@0.5 e6@1.0 a3@0.5 e4@0.5 c6@0.5 d6@1.0 c5@0.5 b4@0.5 e-6@0.5 e6@2.0 a3@0.5 e4@0.5 c5@0.5 a5@1.0 b4@0.5 f6@0.5 d3@0.5 r@2.5 r@3.5 <end>
True
Saving Output file as midi....
-259.4044782295823
Midi Sequnce in text
<start> 50.0@tempo 4/4@time r@3.0 r@3.0 r@3.0 a3@0.5 e4@0.5 c5@0.5 b4@0.5 e6@1.0 a3@0.5 e4@0.5 c6@0.5 d6@1.0 c5@0.5 b4@0.5 e-6@0.5 e6@2.0 a3@0.5 e4@0.5 c5@0.5 a5@1.0 b4@0.5 f6@0.5 d3@0.5 r@2.5 r@3.5 <end>
True
Saving Output file as midi....
7142.123038942926
Midi Sequnce

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Evalution

Function to calculate average tone span (semitones between lowest and highest note in the melody)

In [29]:
def getTonespan(testingSongs):
    cnt = 0
    scale = {"C": 0, "C#": 1, "D-" : 1, "D" : 2, 'D#' : 3, "E-": 3, "E" : 4, "F-": 4, "E#" : 5, "F" : 5, "F#" : 6, "G-" : 6, "G" : 7,
             "G#": 8, "A-": 8, "A": 9, "A#": 10, "B-": 10, "B": 11, "C-":11, "B#": 12 }
    tonespans = []
    for file in testingSongs:
        #print(file)
        try:
            midi = converter.parse(file) #convert to midi
        except:
            print("Error in parsing " + str(file))
            continue
        #tranpose song
        try:
            midi = transpose(midi)
        except:
            print("Error in obtaining key for " + str(file))
            continue
        startBar = getStartingBar(midi)
        if startBar==-1:
            cnt+=1
            continue
        midi = midi.measures(startBar, startBar + 7)
        midi = midi.flat
        minNote = {"value": float('inf'), "name": ''}
        minOct = float('inf')
        maxNote = {"value": float('-inf'), "name": ''}
        maxOct = float('-inf')
        for event in midi:
            if isinstance(event, note.Note):
                if(int(str(event.pitch)[-1]) < minOct):
                    minOct = int(str(event.pitch)[-1])
                if(int(str(event.pitch)[-1]) > maxOct):
                    maxOct = int(str(event.pitch)[-1])
        #print("Min : " + str(minOct) + " Max: " + str(maxOct))
        for event in midi:
            if isinstance(event, note.Note):
                if(int(str(event.pitch)[-1]) == minOct):
                    if( scale[str(event.pitch)[:-1]] < minNote["value"]):
                        minNote["value"] = scale[str(event.pitch)[:-1]]
                        minNote["name"] = event
                if(int(str(event.pitch)[-1]) == maxOct):
                    if( scale[str(event.pitch)[:-1]] > maxNote["value"]):
                        maxNote["value"] = scale[str(event.pitch)[:-1]]
                        maxNote["name"] = event
        #print("MinNote: " + str(minNote["value"]) + " name: " + str(minNote["name"].pitch))
        #print("MaxNote: " + str(maxNote["value"]) + " name: " + str(maxNote["name"].pitch))

        semitones = interval.Interval(minNote["name"], maxNote["name"]).semitones
        #print("semitones: " + str(semitones))
        tonespans.append(int(semitones))

    avg = sum(tonespans)/len(tonespans)
    return avg, cnt


Calculate avg tonespan for all midi files in dataset

In [30]:
# datasetSongs = glob(data_dir + 'midi/*.mid')
datasetSongs = glob('/home/souraja/All_Data_OneAI_Souraja_Final/Image_to_Sequence/testMidi_trans3_VA_loss/*.mid')
print("Avg tone span in dataset: " + str(getTonespan(datasetSongs)))

Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D major Ok
Ok
D m

ZeroDivisionError: division by zero

Calculate avg tonespan for a sample of 16 midi files generated from the model

In [39]:
testingSongs = glob('/home/souraja/All_Data_OneAI_Souraja_Final/Image_to_Sequence/Generated_MIDI/testMidi_trans_enc3_dec3_VA_loss/MIDI100epoch15/*.mid')
cnt, ans = getTonespan(testingSongs)
print("Avg tone span in dataset: " + str(ans))
print("Total escaped music: ",cnt)

Ok
F major Ok
Ok
F major Ok


/tmp/ipykernel_909626/3892802692.py:2: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  cnt, ans = getTonespan(testingSongs)


Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F major Ok
Ok
F m

Function to calculate the scale consistency of a set of midi files

In [32]:
def getAvgScaleConsistency(testingSongs):
    cnt = 0
    A_minor = ["A", "B", "C", "D", 'E', "F", "G#"]
    C_major = ["C", "D", "E", "F", 'G', "A", "B"]

    allRatios = []

    for file in testingSongs:
        try:
            midi = converter.parse(file) #convert to midi
        except:
            print("Error in parsing " + str(file))
            continue
        #tranpose song
        try:
            midi = transpose(midi)
        except:
            print("Error in obtaining key for " + str(file))
            continue

        startBar = getStartingBar(midi)
        if startBar==-1:
            cnt+=1
            continue
        midi = midi.measures(startBar, startBar + 7)
        midi = midi.flat

        correctNotes = 0
        FG_list = []
        GF_list = []
        totNotes = 0
        key = midi.analyze("key")
        if key.mode == "major":
            for event in midi:
                if isinstance(event, note.Note):
                    totNotes += 1
                    if (str(event.pitch)[:-1] in C_major):
                                correctNotes += 1
        elif key.mode == "minor":
            for event in midi:
                if isinstance(event, note.Note):
                    totNotes += 1
                    if((str(event.pitch)[:-1] == "F#") & (len(FG_list) == 0)):
                        FG_list.append("F#")
                        continue
                    if(str(event.pitch)[:-1] == "G#"):
                        if(len(FG_list) == 1):
                            if(FG_list[0] == "F#"):
                                correctNotes += 2
                                FG_list = []
                                continue




                    if((str(event.pitch)[:-1] == "G")  & (len(GF_list) == 0)):
                        GF_list.append("G")
                        continue
                    if(str(event.pitch)[:-1] == "F"):
                        if(len(FG_list) == 1):
                            if(FG_list[0] == "G"):
                                correctNotes += 2
                                GF_list = []
                                continue


                    if (str(event.pitch)[:-1] in A_minor):
                        correctNotes += 1

        allRatios.append(correctNotes/totNotes)

    print("Final avg scale consistency percentage : " + str(sum(allRatios)/len(allRatios) * 100))
    print("Total escaped music", cnt)

Calculate scale consistency of original dataset

In [ ]:
testingSongs = glob('audio/*.mid')
getAvgScaleConsistency(testingSongs)

Calculate scale consistency of testing dataset

In [37]:
testingSongs = glob('/home/souraja/All_Data_OneAI_Souraja_Final/Image_to_Sequence/Generated_MIDI/testMIDI_trans_enc3_dec3/MIDI100epoch15/*.mid')
getAvgScaleConsistency(testingSongs)

Ok
e minor Ok
Ok
e minor Ok


/tmp/ipykernel_909626/2979393780.py:2: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  getAvgScaleConsistency(testingSongs)


Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e minor Ok
Ok
e m

In [3]:
!unzip -j rand_music.zip -d rand_music

Archive:  rand_music.zip
  inflating: rand_music/0001.mid     
  inflating: rand_music/0002.mid     
  inflating: rand_music/0003.mid     
  inflating: rand_music/0004.mid     
  inflating: rand_music/0005.mid     
  inflating: rand_music/0006.mid     
  inflating: rand_music/0007.mid     
  inflating: rand_music/0008.mid     
  inflating: rand_music/0009.mid     
  inflating: rand_music/0010.mid     
  inflating: rand_music/0011.mid     
  inflating: rand_music/0012.mid     
  inflating: rand_music/0013.mid     
  inflating: rand_music/0014.mid     
  inflating: rand_music/0015.mid     
  inflating: rand_music/0016.mid     
  inflating: rand_music/0017.mid     
  inflating: rand_music/0018.mid     
  inflating: rand_music/0019.mid     
  inflating: rand_music/0020.mid     
  inflating: rand_music/0021.mid     
  inflating: rand_music/0022.mid     
  inflating: rand_music/0023.mid     
  inflating: rand_music/0024.mid     
  inflating: rand_music/0025.mid     
  inflating: rand_music/0